In [1]:
import os
import time
import numpy as np
from tqdm import tqdm

import pybullet as p
import pybullet_data
from surrol.utils.pybullet_utils import (
    step,
    get_joints,
    get_link_name,
    reset_camera,
)
from surrol.robots.psm import Psm

pybullet build time: Feb 15 2025 12:06:37


In [2]:
p.connect(p.GUI)
# p.connect(p.DIRECT)
p.setGravity(0, 0, -9.81)

Version = 4.1 Metal - 89.4
Vendor = Apple
Renderer = Apple M1
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


In [3]:
# PsmEnv     : [[ 2.5    3.   ] [-0.25   0.25 ] [ 3.426  3.776]]
# NeedleReach: [[ 2.5    3.   ] [-0.25   0.25 ] [ 3.426  3.776]]

# Actually workspace lowest Z should be 3.476, due to the tray pad.

POSE_TRAY = ((0.55, 0, 0.6751), (0, 0, 0))
SCALING = 10.

In [ ]:
(0.675+0.0102)*5

In [ ]:
obj_id_5 = p.loadURDF('/Users/kantaphat/Research/DEX/SurRoL/surrol/assets/tray/tray_pad.urdf',
                    np.array(POSE_TRAY[0]) * 5.0,
                    p.getQuaternionFromEuler(POSE_TRAY[1]),
                    globalScaling=5.0)

In [ ]:
p.removeBody(obj_id_5)

In [4]:
obj_id_10 = p.loadURDF('/Users/kantaphat/Research/DEX/SurRoL/surrol/assets/tray/tray_pad.urdf',
                    np.array(POSE_TRAY[0]) * 5.0,
                    p.getQuaternionFromEuler(POSE_TRAY[1]),
                    globalScaling=10.0)

In [ ]:
reset_camera(yaw=120, pitch=-25, dist=0.6, target=np.array(POSE_TRAY[0]) * SCALING)

In [9]:
# Front
reset_camera(yaw=90, pitch=-10.0, dist=0.43 * SCALING,
                         target=(-0.05 * SCALING, 0, 0.32 * SCALING))

In [ ]:
# right
reset_camera(yaw=130, pitch=-10.0, dist=0.45 * SCALING,
                         target=(-0.05 * SCALING, 0, 0.34 * SCALING))

In [5]:
POSE_PSM1 = ((0.05, 0.24, 0.8524), (0, 0, -(90 + 20) / 180 * np.pi))
psm = Psm(POSE_PSM1[0], 
          p.getQuaternionFromEuler(POSE_PSM1[1]), 
          scaling=5.0)

In [ ]:
p.removeBody(psm.body)

In [7]:
workspace_limits = np.array([(2.5, 3), (-0.25, 0.25), (3.476, 3.776)])

In [26]:
pos = (workspace_limits[0].mean(),
        workspace_limits[1].mean() + 0.25 * np.cos(np.pi / 3), # 0.25 = radius of hemisphere
        # workspace_limits[2][0] - 0.0102 * SCALING + 0.25 * np.sin(np.pi / 3) - 0.01
        workspace_limits[2][1] + 0.290
        )
orn = (0.5, 0.5, -0.5, -0.5)

joint_positions = psm.inverse_kinematics((pos, orn), psm.EEF_LINK_INDEX)
psm.reset_joint(joint_positions)

array([ 0.62898048, -0.92148253,  0.08409635, -0.36285628,  0.6774394 ,
       -0.89246938])

In [ ]:
workspace_limits[2][1] + 0.290

In [12]:
scale = 50 #25

hemisphere_pos = [
    workspace_limits[0].mean(), 
    workspace_limits[1].mean(), 
    workspace_limits[2][0] - 0.035 # 0.0102 * 2.5
]

print(hemisphere_pos)

# mesh_path = '/Users/kantaphat/Research/DEX/SurRoL/surrol/assets/sphere/half_sphere.urdf'
hemisphere_id = p.loadURDF('/Users/kantaphat/Research/DEX/SurRoL/surrol/assets/sphere/half_sphere.urdf',
            np.array(hemisphere_pos),
            p.getQuaternionFromEuler((-np.pi / 2, 0, 0)),
            # p.getQuaternionFromEuler((-np.pi / 2, 0, np.pi / 2)),
            globalScaling=scale)

[2.75, 0.0, 3.441]


In [ ]:
# View to check if hemisphere is stick to the tray pad
reset_camera(yaw=80, pitch=-0.6, dist=0.37 * SCALING,
                         target=(-0.05 * SCALING, 0, 0.34 * SCALING))

In [11]:
p.removeBody(hemisphere_id)

In [ ]:
np.cos(np.pi / 3)

In [19]:
# Add cylinder

cylinder_pos = [
    workspace_limits[0].mean(), 
    workspace_limits[1].mean() + 0.25 * np.cos(np.pi / 3), 
    workspace_limits[2][1] + 0.15
]

cylinder_visual_id = p.createVisualShape(
    p.GEOM_CYLINDER, 
    radius=0.05, 
    length=0.1,
    rgbaColor=[0, 1, 1, 0.3]
)

cylinder_id = p.createMultiBody(
    baseMass=0,
    baseVisualShapeIndex=cylinder_visual_id,
    basePosition=cylinder_pos,
    baseOrientation=p.getQuaternionFromEuler([-np.pi / 6, 0, 0])
)

In [ ]:
reset_camera(yaw=180, pitch=-89.0, dist=1.0, target=cylinder_pos)

In [18]:
p.removeBody(cylinder_id)

In [ ]:
def draw_workspace(limits, color=(1, 0, 0), line_width=2):
    """Draw a wireframe box representing the workspace limits."""
    x_min, x_max = limits[0]
    y_min, y_max = limits[1]
    z_min, z_max = limits[2]
    
    # Define the 8 corners of the box
    corners = [
        [x_min, y_min, z_min],
        [x_min, y_min, z_max],
        [x_min, y_max, z_min],
        [x_min, y_max, z_max],
        [x_max, y_min, z_min],
        [x_max, y_min, z_max],
        [x_max, y_max, z_min],
        [x_max, y_max, z_max]
    ]
    
    # Define the 12 edges of the box
    edges = [
        (0, 1), (0, 2), (0, 4),
        (1, 3), (1, 5),
        (2, 3), (2, 6),
        (3, 7),
        (4, 5), (4, 6),
        (5, 7),
        (6, 7)
    ]
    
    # Draw each edge
    for edge in edges:
        p.addUserDebugLine(
            corners[edge[0]],
            corners[edge[1]],
            lineColorRGB=color,
            lineWidth=line_width,
            lifeTime=0
        )

In [ ]:
draw_workspace(workspace_limits)

In [ ]:
psm.pose_rcm2world(psm.get_current_position(), 'tuple')[0]

In [ ]:
reset_camera(yaw=90, pitch=-10, dist=1.2, 
             target=np.array(psm.pose_rcm2world(psm.get_current_position(), 'tuple')[0]))

In [ ]:
sphere_shape = p.createVisualShape(
    p.GEOM_SPHERE, 
    radius=0.005, 
    rgbaColor=[0, 1, 1, 1]
)

sphere_body = p.createMultiBody(
    baseMass=0,
    baseVisualShapeIndex=sphere_shape,
    basePosition=psm.pose_rcm2world(psm.get_current_position(), 'tuple')[0]
)

In [ ]:
p.removeBody(sphere_body)

In [27]:
while True:
    p.stepSimulation()
    time.sleep(1.0/240.0)

KeyboardInterrupt: 

In [ ]:
p.disconnect()